<a href="https://colab.research.google.com/github/jitender2622/Learn-pytorch-mnist/blob/main/Learn_pytorch_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt # Added for visualization in Colab

# 1. Device Configuration
# This automatically detects if you have a GPU enabled in Colab
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# 2. Data Pipeline
# Data preprocessing (Standard MNIST normalization values)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load MNIST dataset
# 'download=True' will automatically download the data to Colab's temporary storage
train_dataset = torchvision.datasets.MNIST(
    root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(
    root='./data', train=False, download=True, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# 3. Create the Neural Network
class MNISTClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(784, 128),  # Input: 28x28 = 784 pixels
            nn.ReLU(),
            nn.Linear(128, 10)    # Output: 10 classes (digits 0-9)
        )

    def forward(self, x):
        x = self.flatten(x)
        x = self.layers(x)
        return x

# Initialize model and move to device
model = MNISTClassifier().to(device)

# Loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 4. Training Function
def train_epoch(model, train_loader, loss_function, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        # Move data to the device (CPU or GPU)
        data, target = data.to(device), target.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        output = model(data)

        # Calculate loss
        loss = loss_function(output, target)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        # Track progress
        running_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

        # Print stats every 100 batches
        if batch_idx % 100 == 0 and batch_idx > 0:
            avg_loss = running_loss / 100
            accuracy = 100. * correct / total
            print(f' [Batch {batch_idx}/{len(train_loader)}] '
                  f'Loss: {avg_loss:.3f} | Accuracy: {accuracy:.1f}%')
            running_loss = 0.0

# 5. Evaluation Function
def evaluate(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad(): # Disable gradient calculation for testing
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return 100. * correct / total

# 6. Main Execution Loop
num_epochs = 5  # Reduced to 5 for quicker results in Colab
print("Starting training...")

for epoch in range(num_epochs):
    print(f'\nEpoch: {epoch+1}')
    train_epoch(model, train_loader, loss_function, optimizer, device)
    accuracy = evaluate(model, test_loader, device)
    print(f'End of Epoch {epoch+1} Test Accuracy: {accuracy:.2f}%')

print("\nTraining Finished!")

# --- Optional: Visual Test in Colab ---
# This part grabs a few images from the test set and shows the model's prediction
print("\nRunning a visual test...")
dataiter = iter(test_loader)
images, labels = next(dataiter)
images = images[:5] # Take first 5 images
labels = labels[:5]

# Move to device for prediction
images_dev = images.to(device)
outputs = model(images_dev)
_, predicted = torch.max(outputs, 1)

# Plotting
fig, axes = plt.subplots(1, 5, figsize=(12, 3))
for i in range(5):
    img = images[i].squeeze() # Remove channel dimension for plotting
    axes[i].imshow(img, cmap='gray')
    axes[i].set_title(f"Pred: {predicted[i].item()}\nTrue: {labels[i].item()}")
    axes[i].axis('off')
plt.show()